In [ ]:
# Step 5: Adding plotting to visualize the data
from binance.um_futures import UMFutures
import pandas as pd
import backtrader as bt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

def create_backtrader_feed():
    """
    Create properly formatted data feed for Backtrader
    """
    # Get Binance data
    um_futures_client = UMFutures()
    data = um_futures_client.klines("BTCUSDT", "1d")
    
    columns = [
        "datetime", "open", "high", "low", "close", "volume",
        "Close Time", "Quote Asset Volume", "Number of Trades",
        "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore"
    ]
    
    # Create DataFrame
    df = pd.DataFrame(data, columns=columns)
    
    # Convert datetime properly
    df["datetime"] = pd.to_datetime(df["datetime"], unit='ms')
    
    # Select and convert columns
    df = df[["datetime", "open", "high", "low", "close", "volume"]]
    for col in ["open", "high", "low", "close", "volume"]:
        df[col] = df[col].astype(float)
    
    # Take last 30 days for better visualization
    df = df.tail(30)  # Increased from 10 to 30 for better chart
    
    # Set datetime as index
    df.set_index("datetime", inplace=True)
    
    print("✅ Data prepared for Backtrader:")
    print(f"Date range: {df.index[0]} to {df.index[-1]}")
    print(f"Price range: ${df['close'].min():.2f} - ${df['close'].max():.2f}")
    
    return df

class BinanceData(bt.feeds.PandasData):
    """
    Custom data feed for Binance data
    """
    params = (
        ('datetime', None),
        ('open', 'open'),
        ('high', 'high'),
        ('low', 'low'),
        ('close', 'close'),
        ('volume', 'volume'),
        ('openinterest', -1),
    )

def run_backtest_with_plot():
    """
    Run backtest and display the plot
    """
    # Create Cerebro
    cerebro = bt.Cerebro()
    
    # Get the data
    df = create_backtrader_feed()
    
    # Create data feed
    data_feed = BinanceData(dataname=df)
    
    # Add data to Cerebro
    cerebro.adddata(data_feed)
    
    # Set initial cash
    cerebro.broker.setcash(100000.0)
    
    print(f'\nStarting Portfolio Value: ${cerebro.broker.getvalue():.2f}')
    
    # Run backtest
    cerebro.run()
    
    print(f'Final Portfolio Value: ${cerebro.broker.getvalue():.2f}')
    
    # THIS IS THE KEY LINE FOR PLOTTING
    print("\n📈 Generating plot...")
    cerebro.plot(style='candlestick', barup='green', bardown='red')
    
    print("✅ Plot should appear in a new window!")

if __name__ == '__main__':
    run_backtest_with_plot()